In [ ]:
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import torch
from cmcrameri import cm
from matplotlib.gridspec import GridSpec
from scipy.optimize import curve_fit
from scipy.special import j0
from shapely.geometry import Point, Polygon
from tqdm import tqdm

In [ ]:
plt.style.use(Path("../meta/colorblind_friendly.mplstyle"))
matplotlib.rcParams.update({"font.family": "serif", "font.size": 8})


# station locations
stationlist_all = pl.read_csv("../meta/stations_all.csv")
stations_all = torch.tensor(
    np.vstack((stationlist_all["X"].to_numpy(), stationlist_all["Y"].to_numpy())).T
)

# receiver stations
stationlist_rcv = pl.read_csv("../meta/stations_receivers.csv")
names_receivers = stationlist_rcv["station"]
stations_receivers = torch.tensor(
    np.vstack((stationlist_rcv["X"].to_numpy(), stationlist_rcv["Y"].to_numpy())).T
)

# auxiliary stations
stationlist_aux = pl.read_csv("../meta/stations_auxiliary.csv")
stations_auxiliary = torch.tensor(
    np.vstack((stationlist_aux["X"].to_numpy(), stationlist_aux["Y"].to_numpy())).T
)

master_idx = names_receivers.to_numpy().tolist().index("OMV.GDT")

_cm = 1 / 2.54

fig, ax = plt.subplots(figsize=(1.45 * 9 * _cm, 9 * _cm))
ax.scatter(*stations_all.T, s=5, lw=0, c="#CCC")
ax.scatter(*stations_receivers.T, s=5, lw=0)
ax.scatter(*stations_receivers[master_idx].T, s=100, marker="v", lw=0.5, ec="k")
ax.scatter(*stations_auxiliary.T, s=5, lw=0, c="C4")

ax.set(xlim=(-11, 11), ylim=(-8, 7), aspect="equal")

In [ ]:
# # load correlations
# c1_correlations_filt_both = torch.load(
#     "../data/correlations_for_c1_filt_synth_both.pt", weights_only=False
# )

# c2_correlations_unstacked_filt_both = torch.load(
#     "../data/c2_correlations_unstacked_filt_synth_both.pt", weights_only=False
# )

# # must match compute_correlations.ipynb
# sampling_rate = 5
# length_of_oneside = 300
# window_length = length_of_oneside

# times = torch.arange(0, 2 * length_of_oneside + 1 / sampling_rate, 1 / sampling_rate)
# lapse_times_c1 = torch.arange(
#     -length_of_oneside, length_of_oneside + 1 / sampling_rate, 1 / sampling_rate
# )
# lapse_times_c2 = torch.arange(-window_length / 2, window_length / 2, 1 / sampling_rate)

In [ ]:
# selection based on azimuth

# Zhang et al. 2020 use a fixed angle of 8 degrees, or alpha~0.01
# theta = np.radians(8)
# Nouibat et al. 2022 use a fixed angle of 20 degrees, or alpha~0.06
theta = np.radians(20)

alpha = 1 - np.cos(theta)

# compute a mask for each receiver station (station B)
# that determines which auxiliary stations are included
station_A = stations_receivers[master_idx]
masks_angle = []
for station_B in tqdm(stations_receivers):
    mask = []
    for idx, source in enumerate(stations_auxiliary):
        d_a_b = torch.norm(station_A - station_B)
        d_aux_a = torch.norm(source - station_A)
        d_aux_b = torch.norm(source - station_B)
        include_station = False
        # eq. 14 in the manuscript, eq. 3 of Zhang et al. 2020
        if abs(d_aux_a - d_aux_b) >= (1 - alpha) * d_a_b:
            include_station = True
        mask.append(include_station)
    masks_angle.append(mask)

masks_angle = torch.tensor(masks_angle)

In [ ]:
# select endfire-lobe style
# Roux et al. 2004
f = 0.3
omega = 2 * torch.pi * f
deltaomega = omega * 0.1
c = 1.9

results = []
masks_endfire = []
for station_B in tqdm(stations_receivers):
    # distance_from_center is half distance between the two stations
    a = distance_from_center = torch.norm(station_A - station_B) / 2

    # hacky way of implementing this
    stations = torch.tensor([[-a, 0], [a, 0]], dtype=torch.float32)
    R = 2 * a

    # measure the angle between the two stations
    d_a_b = torch.norm(stations[0] - stations[1])
    dtheta = torch.linspace(-torch.pi / 2, torch.pi / 2, 721)

    # "directivity pattern"
    # eq. 13 in the manuscript, eq. 3 of Roux et al. 2004
    B = 1 - dtheta**4 / 8 * (R / c) ** 2 * (omega**2 + deltaomega**2 / 12)

    # make directivity pattern usable for selection
    # remove dthetas corresponding to B < 0
    dtheta = dtheta[B >= 0]
    B = B[B >= 0]
    # make sure the polygon is large enough
    B *= 2 * 10
    x, y = B * torch.cos(dtheta), B * torch.sin(dtheta)
    # add an x-mirrored copy
    x = torch.cat([-x, x])
    y = torch.cat([y, y])
    # get true theta_0 from the angle between the two stations
    theta_0 = torch.atan2(station_B[1] - station_A[1], station_B[0] - station_A[0])
    # rotate x and y coordinates around center to align with the two stations
    # use numpy for computing trigonometric functions on scalars
    _x = x * np.cos(theta_0) - y * np.sin(theta_0)
    _y = x * np.sin(theta_0) + y * np.cos(theta_0)
    # shift to the center of the two stations
    _x += (station_A[0] + station_B[0]) / 2
    _y += (station_A[1] + station_B[1]) / 2

    polygon = Polygon(zip(_x, _y))
    mask = torch.tensor(
        [polygon.contains(Point(p)) for p in stations_auxiliary], dtype=torch.bool
    )
    masks_endfire.append(mask)

    results.append([_x, _y, mask])

masks_endfire = torch.stack(masks_endfire, dim=0)

In [ ]:
# plot a few examples to see if this works
fig, axs = plt.subplots(
    2, 5, figsize=(2.5 * 9 * _cm, 9 * _cm), sharex=True, sharey=True
)
fig.subplots_adjust(wspace=0, hspace=0.2)
for station_B, mask, ax in zip(stations_receivers, masks_angle, axs[0]):
    ax.scatter(*station_B, marker="v")
    ax.scatter(*station_A, marker="v")
    ax.scatter(*stations_auxiliary[mask].T, marker="o", c="C4", s=5, lw=0)
    ax.set(xlim=(-6, 6), ylim=(-6, 6), aspect="equal")

for idx, (ax, r) in enumerate(zip(axs[1], results)):
    ax.scatter(*stations_receivers[idx], marker="v")
    ax.scatter(*station_A, marker="v")
    ax.scatter(*stations_auxiliary[r[-1]].T, marker="o", c="C4", s=5, lw=0)
    # ax.plot(*r[:2], alpha=0.25, c="k")
    ax.set(xlim=(-6, 6), ylim=(-6, 6), aspect="equal")

In [ ]:
# balance number of auxiliary stations on either side of the station pair to avoid asymmetry bias
# for each station pair and determine the "sidedness"  for all auxiliary stations


def is_aux_station_left(station_A, station_B, station):
    """Determine if the auxiliary station is on the left or right side."""
    AB = station_B - station_A
    mid_point = (station_A + station_B) / 2
    perp_vector = torch.tensor([-AB[1], AB[0]])
    AS_mid = station - mid_point
    cross_product = perp_vector[0] * AS_mid[1] - perp_vector[1] * AS_mid[0]
    return True if cross_product > 0 else False


aux_station_classification = []
for station_B in tqdm(stations_receivers):
    is_left = []
    for station in stations_auxiliary:
        is_left.append(is_aux_station_left(station_A, station_B, station))
    aux_station_classification.append(is_left)
aux_station_classification = torch.tensor(aux_station_classification, dtype=torch.bool)

In [ ]:
# balance masks to have the same number of stations on either side
masks_angle_balanced = masks_angle.clone()
for idx, mask in enumerate(masks_angle_balanced):
    # determine number of stations on either side of the station pair
    n_left_stations = torch.sum(aux_station_classification[idx][mask])
    n_right_stations = torch.sum(~aux_station_classification[idx][mask])
    n_to_remove = abs(n_left_stations - n_right_stations)

    if n_left_stations > n_right_stations:
        # find indices of left stations
        is_left = aux_station_classification[idx]
        # of all stations that for which is_left and mask are True,
        # remove n_to_remove random indices
        left_station_idxs = torch.where(is_left & mask)[0]
        remove_idxs = torch.randperm(len(left_station_idxs))[:n_to_remove]
        mask[left_station_idxs[remove_idxs]] = False
    elif n_right_stations > n_left_stations:
        # find indices of right stations
        is_right = ~aux_station_classification[idx]
        # of all stations that for which is_right and mask are True,
        # remove n_to_remove random indices
        right_station_idxs = torch.where(is_right & mask)[0]
        remove_idxs = torch.randperm(len(right_station_idxs))[:n_to_remove]
        mask[right_station_idxs[remove_idxs]] = False


masks_endfire_balanced = masks_endfire.clone()
for idx, mask in enumerate(masks_endfire_balanced):
    n_left_stations = torch.sum(aux_station_classification[idx][mask])
    n_right_stations = torch.sum(~aux_station_classification[idx][mask])
    n_to_remove = abs(n_left_stations - n_right_stations)

    if n_left_stations > n_right_stations:
        # find indices of left stations
        is_left = aux_station_classification[idx]
        # of all stations that for which is_left and mask are True,
        # remove n_to_remove random indices
        left_station_idxs = torch.where(is_left & mask)[0]
        remove_idxs = torch.randperm(len(left_station_idxs))[:n_to_remove]
        mask[left_station_idxs[remove_idxs]] = False
    elif n_right_stations > n_left_stations:
        # find indices of right stations
        is_right = ~aux_station_classification[idx]
        # of all stations that for which is_right and mask are True,
        # remove n_to_remove random indices
        right_station_idxs = torch.where(is_right & mask)[0]
        remove_idxs = torch.randperm(len(right_station_idxs))[:n_to_remove]
        mask[right_station_idxs[remove_idxs]] = False

# plot a few examples to see if this works
fig, axs = plt.subplots(
    2, 5, figsize=(2.5 * 9 * _cm, 9 * _cm), sharex=True, sharey=True
)
fig.subplots_adjust(wspace=0, hspace=0.2)
for ax, station_B, mask in zip(axs[0], stations_receivers, masks_angle_balanced):
    # ax.scatter(*stations_aux[mask].T, marker="o", s=5, lw=0, c="#94A4A2")
    ax.scatter(*station_B, marker="v")
    ax.scatter(*station_A, marker="v")
    ax.scatter(*stations_auxiliary[mask].T, marker="o", c="C4", s=5, lw=0)
    ax.set(xlim=(-6, 6), ylim=(-6, 6), aspect="equal")
for ax, station_B, mask in zip(axs[1], stations_receivers, masks_endfire_balanced):
    ax.scatter(*station_B, marker="v")
    ax.scatter(*station_A, marker="v")
    ax.scatter(*stations_auxiliary[mask].T, marker="o", c="C4", s=5, lw=0)
    ax.set(xlim=(-6, 6), ylim=(-6, 6), aspect="equal")

In [ ]:
# load correlations
c1_correlations_filt_both = torch.load(
    "../data/correlations_for_c1_filt_synth_both.pt", weights_only=False
)
c1_correlations_filt_boundary = torch.load(
    "../data/correlations_for_c1_filt_synth_boundary.pt", weights_only=False
)

c2_correlations_unstacked_filt_both = torch.load(
    "../data/c2_correlations_unstacked_filt_synth_both.pt", weights_only=False
)

# must match compute_correlations.ipynb
sampling_rate = 5
length_of_oneside = 300
window_length = length_of_oneside

times = torch.arange(0, 2 * length_of_oneside + 1 / sampling_rate, 1 / sampling_rate)
lapse_times_c1 = torch.arange(
    -length_of_oneside, length_of_oneside + 1 / sampling_rate, 1 / sampling_rate
)
lapse_times_c2 = torch.arange(-window_length / 2, window_length / 2, 1 / sampling_rate)

In [ ]:
# average causal and anti-causal C2 correlations
c2_correlations_unstacked_filt_both_avg = c2_correlations_unstacked_filt_both.mean(
    dim=0
)

# auxiliary station averaging
# all-direction selection.
c2_correlations_filt_both_allstack = c2_correlations_unstacked_filt_both_avg.mean(dim=1)

# angle selection
c2_correlations_filt_both_anglestack = torch.zeros_like(
    c2_correlations_filt_both_allstack
)
for i, mask in enumerate(masks_angle_balanced):
    c2_correlations_filt_both_anglestack[i] = c2_correlations_unstacked_filt_both_avg[
        i
    ][mask].mean(dim=0)

# endfire-lobe selection
c2_correlations_filt_both_endfirestack = torch.zeros_like(
    c2_correlations_filt_both_allstack
)
for i, mask in enumerate(masks_endfire_balanced):
    c2_correlations_filt_both_endfirestack[i] = c2_correlations_unstacked_filt_both_avg[
        i
    ][mask].mean(dim=0)

In [ ]:
c2_zerolag_index = lapse_times_c2.abs().argmin()
c2_focal_spots = torch.stack(
    [
        c2_correlations_filt_both_allstack[..., c2_zerolag_index],
        c2_correlations_filt_both_endfirestack[..., c2_zerolag_index],
        c2_correlations_filt_both_anglestack[..., c2_zerolag_index],
    ]
)

# replace all nans with 0s
c2_focal_spots[torch.isnan(c2_focal_spots)] = 0.0

for fs in c2_focal_spots:
    fs /= fs.abs().max()

c1_zerolag_index = lapse_times_c1.abs().argmin()
c1_focal_spots = torch.stack(
    [
        c1_correlations_filt_boundary[..., c1_zerolag_index],
        c1_correlations_filt_both[..., c1_zerolag_index],
    ]
)
c1_focal_spots[torch.isnan(c1_focal_spots)] = 0.0

for fs in c1_focal_spots:
    fs /= fs.abs().max()


In [ ]:
def bessel_func(r, a, v):
    f = 0.3
    return a * j0(r * 2 * np.pi * f / v)


def iso_focal_spot_fit(focal_spot, distances):
    # initial guess for a and v
    initial_guess = [np.random.uniform(0.5, 1.5), np.random.uniform(1.25, 2.75)]

    # fit the bessel function to the focal spot data
    popt, _ = curve_fit(
        bessel_func,
        distances.numpy(),
        focal_spot.numpy(),
        p0=initial_guess,
    )
    a_fit, v_fit = popt

    # compute the fitted focal spot data
    focal_spot_fit = bessel_func(distances.numpy(), a_fit, v_fit)
    focal_spot_fit = torch.tensor(focal_spot_fit)
    # normalize the fitted focal spot data
    focal_spot_fit /= focal_spot_fit.abs().max()

    return focal_spot_fit, v_fit


# compute distances from master station
distances = torch.norm(stations_receivers - stations_receivers[master_idx], dim=1)
max_dist_for_fit = 4.5
mask_for_fitting = distances <= max_dist_for_fit

c2_focal_spots_masked = c2_focal_spots[:, mask_for_fitting]
c1_focal_spots_masked = c1_focal_spots[:, mask_for_fitting]

focal_spot_fits = []
v_fits = []
for fs in c1_focal_spots_masked:
    focal_spot_fit, v_fit = iso_focal_spot_fit(
        focal_spot=fs,
        distances=distances[mask_for_fitting],
    )
    focal_spot_fits.append(focal_spot_fit)
    v_fits.append(v_fit)
for fs in c2_focal_spots_masked:
    focal_spot_fit, v_fit = iso_focal_spot_fit(
        focal_spot=fs,
        distances=distances[mask_for_fitting],
    )
    focal_spot_fits.append(focal_spot_fit)
    v_fits.append(v_fit)

all_focal_spots = torch.cat([c1_focal_spots, c2_focal_spots], dim=0)

In [ ]:
fig = plt.figure(figsize=(18 * _cm, 0.45 * 18 * _cm))
gs = GridSpec(
    2,
    5,
    figure=fig,
    height_ratios=[1, 0.7],
    width_ratios=[1, 1, 1, 1, 1],
    hspace=0.1,
    wspace=0.1,
)
axs = np.array([fig.add_subplot(g) for g in gs]).reshape(2, 5)


lbls = ["a", "b", "c", "d", "e"]
descriptions = [
    r"$C^{ideal}_1$",
    r"$C_1$",
    r"$C^{all}_2$",
    r"$C^{endfire}_2$",
    r"$C^{angle}_2$",
]
for ax, fs, lbl, description in zip(
    axs[:-1, :].flat, all_focal_spots, lbls, descriptions
):
    sct = ax.scatter(
        *stations_receivers.T,
        c=fs,
        cmap=cm.broc,
        s=5,
        vmin=-1,
        vmax=1,
        lw=0,
    )
    # circle with radius 4.5 km
    circle = plt.Circle(
        stations_receivers[master_idx],
        max_dist_for_fit,
        color="k",
        fill=False,
        ls="--",
        lw=0.75,
        # alpha=0.5,
    )
    ax.add_artist(circle)

    ax.set(
        aspect="equal",
        xlim=(-4.5, 4.5),
        ylim=(-4.5, 4.5),
        xticks=(-4, 0, 4),
        yticks=(-4, 0, 4),
    )
    ax.set_xlabel("Distance [km]", labelpad=0, fontsize=8)
    ax.set_xticklabels([-4, 0, 4], fontsize=8)
    ax.set_yticklabels([-4, 0, 4], fontsize=8)

    ax.scatter(
        *stations_receivers[master_idx].T,
        marker="v",
        c="#FFA90E",
        ec="k",
        s=75,
        lw=0.5,
    )

    ax.set_title(
        f"{lbl})",
        loc="left",
        fontsize=10,
        pad=4,
    )

    ax.text(
        0.05,
        0.95,
        description,
        ha="left",
        va="top",
        transform=ax.transAxes,
        fontsize=8,
        fontweight="bold",
        bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.7),
    )

    # on last, colorbar
    if ax == axs[0, -1]:
        x0, y0, w, h = ax.get_position().bounds
        cbar_ax = fig.add_axes([x0 + w + 0.005, y0, 0.0075, h])
        cbar = fig.colorbar(sct, cax=cbar_ax, ticks=[-1, 0, 1])
        cbar.ax.tick_params(labelsize=8)
        cbar.set_label("Simulated\namplitudes", fontsize=8, labelpad=-5)

axs[0, 0].set_ylabel("Distance [km]", labelpad=0, fontsize=8)
for ax in axs[:, 1:].flat:
    ax.set_yticklabels([])

## amplitude-vs-dist plots
lbls = ["f", "g", "h", "i", "j"]
for ax, fs, v_fit, lbl in zip(
    axs[-1, :].flat,
    all_focal_spots,
    v_fits,
    lbls,
):
    # compute backazimuth of all stations from master
    backazimuths = torch.atan2(
        stations_receivers[:, 1] - stations_receivers[master_idx][1],
        stations_receivers[:, 0] - stations_receivers[master_idx][0],
    )

    # set direction in NW (0.8*pi) to be 0 degrees
    backazimuths -= 0.8 * torch.pi
    # wrap to [0, 2pi]
    backazimuths = backazimuths % (2 * torch.pi)
    # symmetrize [-pi, pi]
    backazimuths = (backazimuths + torch.pi) % (2 * torch.pi) - torch.pi
    clrs = cm.lipari_r(1 - torch.cos(backazimuths).abs())

    sct = ax.scatter(
        distances,
        fs,
        s=3,
        c=clrs,
        lw=0,
    )

    # plot the fitted bessel function
    r = torch.linspace(0, 5, 100)
    res = bessel_func(r, 1, v_fit)
    ax.plot(r, res, c="w", lw=3)
    ax.plot(r, res, c="k", lw=1.5)

    # plot v_fit value
    ax.text(
        0.98,
        0.95,
        f"c = {round(v_fit, 2):.2f} km/s",
        ha="right",
        va="top",
        transform=ax.transAxes,
        fontsize=7,
        bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.7),
    )

    ax.set(
        xlim=(0, 5),
        ylim=(-1.1, 1.1),
    )
    ax.set_xlabel("Distance to    [km]", labelpad=0, fontsize=8)
    ax.set_xticks([0, 2, 4])
    ax.set_xticklabels([0, 2, 4], fontsize=8)
    ax.set_xlim(0, 4.5)
    ax.set_yticks([-1, 0, 1])
    ax.set_title(
        f"{lbl})",
        loc="left",
        fontsize=10,
        pad=4,
    )

    # indicate master as icon in xlabel text where the gap is
    ax.scatter(
        0.68,
        -0.2,
        marker="v",
        c="#FFA90E",
        ec="k",
        s=30,
        lw=0.5,
        clip_on=False,
        transform=ax.transAxes,
    )

    ax.axhline(0, c="#CCC", lw=1, ls="--", zorder=-1)

    # move ax down a bit
    x0, y0, w, h = ax.get_position().bounds
    ax.set_position([x0, y0 - 0.05, w, h])  # move down by 0.02

    # custom circular cbar.
    if ax == axs[-1, 0]:
        # draw a full ring colored by backazimuths into a new axis (roughly quarter size of ax)
        x0, y0, w, h = ax.get_position().bounds
        ring_ax = fig.add_axes([x0 + 0.02, y0 + h / 20, w / 2.5, h / 2.5], polar=True)
        ring_ax.axis("off")
        n_slices = 73
        theta = torch.linspace(0, 2 * torch.pi, n_slices)
        r = 1.0
        x = r * torch.cos(theta)
        y = r * torch.sin(theta)
        colors = cm.lipari_r(1 - torch.cos(theta - 0.8 * torch.pi).abs())
        ring_ax.bar(
            theta,
            r,
            width=torch.pi * 2 / n_slices,
            bottom=0.5,
            facecolor=colors,
            edgecolor="None",
            lw=0,
        )
        # add inner and outer circle
        ring_ax.plot(
            theta,
            0.5 * torch.ones_like(theta),
            c="k",
            lw=0.75,
        )
        ring_ax.plot(
            theta,
            1.5 * torch.ones_like(theta),
            c="k",
            lw=0.75,
        )
        # indicate direction towards isolated sources with arrow
        ring_ax.annotate(
            "",
            xy=(0.8 * torch.pi, 1.5),
            xytext=(0.8 * torch.pi, 3.2),
            arrowprops=dict(facecolor="black", arrowstyle="<-", lw=1),
        )
        ring_ax.text(
            0.8 * torch.pi,
            2,
            r"$\mathbf{r}_I$",
            ha="left",
            va="bottom",
            fontsize=8,
        )

axs[-1, 0].set_yticklabels([-1, 0, 1], fontsize=8)
axs[-1, 0].set_ylabel("Amplitude", labelpad=0, fontsize=8)

fig.savefig("../figures/figure8.png", dpi=300, bbox_inches="tight")